# Calibrating the the Incubator Temperature Dynamics

## Step 1: Get the calibration data

The data comes from an experiment where the heater was turned on, letting the incubator warm up for some time, and then turned off, before repeating.

In [ ]:
# Get CSV file from an incubator dataset
import os

# Get the current working directory.
current_dir = os.getcwd()

assert os.path.basename(current_dir) == '4-Calibration', 'Current directory is not 4-Calibration'

# Get the parent directory. Should be the root of the repository
parent_dir = os.path.dirname(current_dir)

# The root of the repo should contain the incubator_dt folder. Otherwise something went wrong in 0-Pre-requisites.
assert os.path.exists(os.path.join(parent_dir, 'incubator_dt')), 'incubator_dt folder not found in the repository root'

csv_file_path = os.path.join(parent_dir, 'incubator_dt', 'software', 'incubator', 'datasets', '20230501_calibration_empty_system', '20230501_calibration_empty_system.csv')

assert os.path.exists(csv_file_path), '20230501_calibration_empty_system.csv not found in the incubator repository.'

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

data = pd.read_csv(csv_file_path)
data.head()

In [ ]:
# Create subplots with shared X-axis for temperature and control signal.
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 6))

# Plot temperature data on the first subplot
ax1.plot(data.index, data.average_temperature, label='Temperature')
ax1.plot(data.index, data.t3, label='Room Temperature')
ax1.set_ylabel('Temperature (°C)')
ax1.legend()

# Plot control signal data on the second subplot (heater on/off)
ax2.plot(data.index, data.heater_on, label='Heater')
ax2.set_xlabel('Samples')
ax2.set_ylabel('Control Signal')
ax2.legend()

# Add a title to the shared X-axis
fig.suptitle('Temperature and Control Signal Over Time')

# Show the plots
plt.show()

## Step 2: Define the ODE System for the Incubator

We reuse the model created in [3-Physics-Modelling\2-ModellingIncubatorDynamics.ipynb](..\3-Physics-Modelling\2-ModellingIncubatorDynamics.ipynb), extended with the interpolated continuous functions from the dataset.

In [ ]:
# Example of a varying power and room temperature
from scipy.interpolate import interp1d
import numpy as np

# Create lookup functions for model inputs from the data.
time_range = data["time"].to_numpy().astype(np.float64)

# Time has been stored in nanoseconds, so we need to convert it to seconds
time_range = time_range / 1e9

# interp1d returns a function of time, 
#  that finds the closest value in the data to the time given.
# This is useful for interpolating the data to the time steps of the model, when estimating the model parameters.
heater_on_lookup_function = interp1d(time_range, data["heater_on"].to_numpy())
T_room_lookup_function = interp1d(time_range, data["t3"].to_numpy())

# Plot interpolated data vs original data. It should be fairly close.
fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 6))
# Plot temperature data on the first subplot
ax1.plot(time_range, data.t3, label='Room Temperature')
ax1.plot(time_range, [T_room_lookup_function(t) for t in time_range], label='Room Temperature (Interopolated)')
ax1.set_ylabel('Temperature (°C)')
ax1.legend()

# Plot control signal data on the second subplot (heater on/off)
ax2.plot(time_range, data.heater_on, label='Heater')
ax2.plot(time_range, [heater_on_lookup_function(t) for t in time_range], label='Heater (Interopolated)')
ax2.set_xlabel('Time')
ax2.set_ylabel('Control Signal')
ax2.legend()

# Add a title to the shared X-axis
fig.suptitle('Temperature and Control Signal Over Time')

# Show the plots
plt.show()

In [ ]:
from scipy.integrate import solve_ivp

# ODE system
def incubator_ode_with_varying_conditions(t, y, Ch, Cb, G_hb, G_br, Voltage, Current):
    Th, Tb = y
    Ph = Voltage*Current  # Power in (W)
    Tr = T_room_lookup_function(t)  # Interpolated room temperature at time t
    H_h = heater_on_lookup_function(t)  # Heater on/off switch at time t
    dTh_dt = (H_h * Ph - G_hb * (Th - Tb)) / Ch
    dTb_dt = (G_hb * (Th - Tb) - G_br * (Tb - Tr)) / Cb
    return [dTh_dt, dTb_dt]

# Parameters (arbitrary values for demonstration purposes)
Ch = 500.0  # Heat capacity of the heater (J/Kg/C)
Cb = 1000.0  # Heat capacity of the box (J/Kg/C)
G_hb = 10.0  # Heat transfer coefficient between heater and box (W/C)
G_br = 5.0  # Heat transfer coefficient between box and room (W/C)
Voltage = 12.0  # Voltage of the Heater (V)
Current = 1.5  # Current of the Heater (A)

# Initial conditions
T_h0 = data.average_temperature[0]  # Initial heater temperature (C). Assumed to be the same as the initial box temperature and comes from initial value of the data.
T_b0 = data.average_temperature[0]  # Initial box temperature (C)
y0 = [T_h0, T_b0]  # Initial state vector

# Time span: same as the dataset
t_span = (time_range[0], time_range[-1])  # Start and end times
t_eval = time_range  # Times at which to evaluate the solution. Should match the dataset.

sol = solve_ivp(
        lambda t, y: incubator_ode_with_varying_conditions(t, y, Ch, Cb, G_hb, G_br, Voltage, Current),
        t_span, y0, t_eval=t_eval)

sol

In [ ]:
# Plot the results
Th_values = sol.y[0,:]
Tb_values = sol.y[1,:]

plt.figure(figsize=(10, 6))
plt.plot(data.time, Th_values, color='blue', linestyle="--", label='Heater Temperature Simulated (Th)')
plt.plot(data.time, data.average_temperature, color='red', label='Box Temperature from data (Tb)')
plt.plot(data.time, Tb_values, color='red', linestyle="--", label='Box Temperature Simulated (Tb)')
plt.title('Temperature Evolution in Incubator')
plt.xlabel('Time (seconds)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Now let us try out different values of some of the ODE parameters. Our goal is to make the dashed red line (simulated temperature of the box) match the continuous red line (temperature from the dataset).

In [ ]:
# Try changing the following values. As a first suggestion, we've decreased the values of the G_hb and G_br parameter.
Ch = 500.0  # Heat capacity of the heater (J/Kg/C)
Cb = 1000.0  # Heat capacity of the box (J/Kg/C)
G_hb = 1.0  # Heat transfer coefficient between heater and box (W/C)
G_br = 1.0  # Heat transfer coefficient between box and room (W/C)
Voltage = 12  # Voltage of the Heater (V)
Current = 1.5  # Current of the Heater (A)

sol = solve_ivp(
        lambda t, y: incubator_ode_with_varying_conditions(t, y, Ch, Cb, G_hb, G_br, Voltage, Current),
        t_span, y0, t_eval=t_eval)

Th_values = sol.y[0,:]
Tb_values = sol.y[1,:]

plt.figure(figsize=(10, 6))
# Omit heater temperature as not important since we do not have heater temperature data
# plt.plot(data.time, Th_values, color='blue', label='Heater Temperature (Th)')
plt.plot(data.time, data.average_temperature, color='red', label='Box Temperature (Tb)')
plt.plot(data.time, Tb_values, color='red', linestyle="--", label='Box Temperature Simulated (Tb)')
plt.title('Temperature Evolution in Incubator')
plt.xlabel('Time (seconds)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Try changing the above parameters until the box temperature simulated matches the box temperature, or gets closer. You are effectively doing what the optimization algorithm will do.

## Step 2: Define Cost Function for the Optimization

The cost function is a function of the parameters, and should return a scalar value reflecting the fitness of those parameters. The strategy is, for each valuation of the parameters, run a simulation, and then compute the sum of squared errors between the simulated temperature and the temperature in the data.

In [ ]:
# Convert average_temperature to numpy array for use in the cost function, for better efficiency.
average_temperature = data.average_temperature.to_numpy()

def cost(P_guess):
    Ch,Cb,G_hb,G_br,Voltage,Current = P_guess
    
    # Note that this part of the function depends on the initial values t_span, y0, and t_eval, defined in the previous cells. 
    sol = solve_ivp(
        lambda t, y: incubator_ode_with_varying_conditions(t, y, Ch, Cb, G_hb, G_br, Voltage, Current),
        t_span, y0, t_eval=t_eval)

    # Check if simulation was successful
    if not sol.success:
        print(f"Cost for {P_guess}: Simulation failed")
        return 1e6  # Return a high cost to avoid this solution
    else:
        Tb_values = sol.y[1,:]
        differences = average_temperature - Tb_values
        sum_sq_dff = sum(differences**2)
        print(f"Cost for {P_guess}: {sum_sq_dff}")
        return sum_sq_dff

# Try different values for the parameters
#                Ch,       Cb,  G_hb, G_br,Voltage,Current
initial_guess = [500.0, 1000.0, 10.0, 5.0,  12.0,   1.5]
cost(initial_guess)

initial_guess = [300.0, 200.0, 1.0, 0.57,  12.0,   1.5]
cost(initial_guess)

## Step 3: Run the Optimization

In [ ]:
from scipy.optimize import least_squares

initial_guess = [300.0, 200.0, 1.6, 0.57,  12.0,   1.5]
res = least_squares(cost, initial_guess)
res

Depending on the initial guess selected you may or may not be successful at finding a minimum for the optimization problem. It may help to get an initial guest that is as close as possible to the minimum and possibly incorporate constraints on the values of parameters based on your knowledge of the physics. For instance, all parameters must be at least positive values.

The following does just that. We know from literature and datasheets of the components what the possible bounds for the parameters are, so we define those.

In [ ]:
#       Ch,     Cb,    G_hb,   G_br,   Voltage, Current
bounds=([10.0,  10.0,  0.0,    0.0,    12.0,    1.0], 
        [500.0, 500.0, 5.0,    5.0,    13.0,    2.0])

initial_guess = [300.0, 200.0, 1.6, 0.57,  12.0,   1.5]

res = least_squares(cost, initial_guess, bounds=bounds)
res

Now we're more likely to successully converge on a minimum.

In [ ]:
# Print optimal parameters:
Ch_opt, Cb_opt, G_hb_opt, G_br_opt, Voltage_opt, Current_opt = res.x
print(f"Optimal parameters:")
print(f"Ch: {Ch_opt}")
print(f"Cb: {Cb_opt}")
print(f"G_hb: {G_hb_opt}")
print(f"G_br: {G_br_opt}")
print(f"Voltage: {Voltage_opt}")
print(f"Current: {Current_opt}")

## Step 4: Plot Results

In [ ]:
optimal_parameters = res.x
Ch,Cb,G_hb,G_br,Voltage,Current = optimal_parameters

sol = solve_ivp(
        lambda t, y: incubator_ode_with_varying_conditions(t, y, Ch, Cb, G_hb, G_br, Voltage, Current),
        t_span, y0, t_eval=t_eval)

Th_values = sol.y[0,:]
Tb_values = sol.y[1,:]

plt.figure(figsize=(10, 6))
# Omit heater temperature as not important since we do not have heater temperature data
plt.plot(data.time, Th_values, color='blue', label='Heater Temperature (Th)')
plt.plot(data.time, data.average_temperature, color='red', label='Box Temperature (Tb)')
plt.plot(data.time, Tb_values, color='red', linestyle="--", label='Box Temperature Simulated (Tb)')
plt.title('Temperature Evolution in Incubator')
plt.xlabel('Time (seconds)')
plt.ylabel('Temperature (°C)')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

Note how the calibration is not perfect. This is often the case, since our model simplifies the physics by quite a bit!

Also note how we now have a way to predict the heater temperature, even though we do not have a sensor for it. Our calibration would be improved if we had sensor data for the heater temperature, but we can be reasonably confident in our predictions of it, since the box temperatures are aligned.

## Exercises

1. Go back to the notebook [3-Physics-Modelling\2-ModellingIncubatorDynamics.ipynb](../3-Physics-Modelling\2-ModellingIncubatorDynamics.ipynb) and adjust the parameters of the model there, so that you can get a more realistic impression of how the controller works. Then adjust the controller parameters to ensure it works better (e.g., does not overshoot the maximum or undershoot the minimum temperature so much.)
2. Compare the above calibration code with the training of the neural network in [3-Physics-Modelling\4-TrainingIncubatorNN.ipynb](../3-Physics-Modelling\4-TrainingIncubatorNN.ipynb). What are the similarities and differences?
3. Just like in training of the neural network, we should have a training/test split of the data. Adjust the above code to do just that.
4. Other than obtaining the heater sensor data, what are some other actions that we could take to try to improve our calibration?
5. Do you think that it is an issue that our simulated box temperature is not perfect? Relate your explanation to the services provided by the DT.
6. Let's try a different dataset.
   a. Record the optimal parameters for this original dataset.
   b. Change the dataset file to `incubator_dt/software/incubator/datasets/20230501_tempeh_batch/rec_2023-05-01__08_09_06.csv`, and rerun the steps of the notebook.. This is a run of the incubator with tempeh fermenting inside.
   b. Compare the old optimal parameters with the new optimal parameters. Explain the differences and similarities.